In [2]:
import warnings

warnings.filterwarnings("ignore", category=UserWarning, module="keras.layers.rnn.rnn")

#### Data Preprocessing

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\s4114805\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\s4114805\AppData\Roaming\nltk_data...


In [4]:
df = pd.read_csv('My_Thesis_Data.csv')
df

,Unnamed: 0,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,0,04/25/23,"Credit reporting, credit repair services, or o...",Credit reporting,Improper use of your report,Reporting company used your report improperly,My name is XXXX XXXX XXXX this complaint is no...,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,MD,21206,None,Consent provided,Web,04/25/23,Closed with explanation,Yes,NaN,6880772
1,1,07/10/23,"Credit reporting, credit repair services, or o...",Credit reporting,Improper use of your report,Reporting company used your report improperly,In accordance with the Fair Credit Reporting a...,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",OK,73115,None,Consent provided,Web,07/10/23,Closed with non-monetary relief,Yes,NaN,7229816
2,2,01/10/24,Credit reporting or other personal consumer re...,Credit reporting,Incorrect information on your report,Account information incorrect,By the Laws that uphold deemed as FCRA ( Fair ...,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",CA,91606,None,Consent provided,Web,01/10/24,Closed with non-monetary relief,Yes,NaN,8139386
3,3,08/01/23,Checking or savings account,Checking account,Opening an account,Didn't receive terms that were advertised,I am inputting a new compliant in follow up fr...,Company has responded to the consumer and the ...,"CITIBANK, N.A.",TX,75126,None,Consent provided,Web,08/01/23,Closed with explanation,Yes,NaN,7334854
4,4,02/27/23,Mortgage,Conventional home mortgage,Trouble during payment process,None,Mortgage Concern : I received a mortgage in XX...,None,SN SERVICING CORPORATION,DC,20003,None,Consent provided,Web,03/22/23,Closed with explanation,Yes,NaN,6625444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
542644,542644,03/04/23,Debt collection,Mortgage debt,Attempts to collect debt not owed,Debt was paid,Rent relief paid the outstanding balance of XX...,None,"National Credit Systems,Inc.",TX,76040,None,Consent provided,Web,03/04/23,Closed with explanation,Yes,NaN,6646662
542645,542645,04/06/23,Debt collection,Other debt,Attempts to collect debt not owed,Debt was already discharged in bankruptcy and ...,Have the information removed,None,CL Holdings LLC,SC,29360,None,Consent provided,Web,04/06/23,Closed with explanation,Yes,NaN,6803727
542646,542646,08/11/23,Debt collection,Credit card debt,Communication tactics,Frequent or repeated calls,Upgrade continues to call me over and over Mut...,None,"Upgrade, Inc.",FL,33065,None,Consent provided,Web,08/11/23,Closed with explanation,Yes,NaN,7389495
542647,542647,02/24/23,"Credit reporting, credit repair services, or o...",Credit reporting,Problem with a credit reporting company's inve...,Their investigation did not fix an error on yo...,Originally I tried to dispute with the credit ...,None,"EQUIFAX, INC.",KY,42141,None,Consent provided,Web,02/24/23,Closed with explanation,Yes,NaN,6606585


In [5]:
df.columns

Index(['Unnamed: 0', 'Date received', 'Product', 'Sub-product', 'Issue',
       'Sub-issue', 'Consumer complaint narrative', 'Company public response',
       'Company', 'State', 'ZIP code', 'Tags', 'Consumer consent provided?',
       'Submitted via', 'Date sent to company', 'Company response to consumer',
       'Timely response?', 'Consumer disputed?', 'Complaint ID'],
      dtype='object')

In [6]:
df.isnull().sum()

Unnamed: 0                           0
Date received                        0
Product                              0
Sub-product                          0
Issue                                0
Sub-issue                            0
Consumer complaint narrative         0
Company public response              0
Company                              0
State                                0
ZIP code                             0
Tags                                 0
Consumer consent provided?           0
Submitted via                        0
Date sent to company                 0
Company response to consumer         0
Timely response?                     0
Consumer disputed?              542649
Complaint ID                         0
dtype: int64

In [7]:
# Get data types for each column
df_dtypes = df.dtypes

# Add comment explaining the meaning
print("Data types of each column in the DataFrame:\n", df_dtypes)

Data types of each column in the DataFrame:
 Unnamed: 0                        int64
Date received                    object
Product                          object
Sub-product                      object
Issue                            object
Sub-issue                        object
Consumer complaint narrative     object
Company public response          object
Company                          object
State                            object
ZIP code                         object
Tags                             object
Consumer consent provided?       object
Submitted via                    object
Date sent to company             object
Company response to consumer     object
Timely response?                 object
Consumer disputed?              float64
Complaint ID                      int64
dtype: object


In [8]:
# Access the 'Consumer complaint narrative' column
consumer_complaints = df['Consumer complaint narrative']

# Add comment explaining the content of the column
print("This column contains the text of consumer complaints:",
      consumer_complaints)

This column contains the text of consumer complaints: 0         My name is XXXX XXXX XXXX this complaint is no...
1         In accordance with the Fair Credit Reporting a...
2         By the Laws that uphold deemed as FCRA ( Fair ...
3         I am inputting a new compliant in follow up fr...
4         Mortgage Concern : I received a mortgage in XX...
                                ...                        
542644    Rent relief paid the outstanding balance of XX...
542645                         Have the information removed
542646    Upgrade continues to call me over and over Mut...
542647    Originally I tried to dispute with the credit ...
542648    On XX/XX/XXXX, I opened a Smartly Checking acc...
Name: Consumer complaint narrative, Length: 542649, dtype: object


In [9]:
# Special character removal
def clean_text(text):
  """
  This function cleans text data by removing special characters, punctuations, numbers,
  uppercase 'X', and convert it to lowercase.

  Args:
      text (str): The text to be cleaned.

  Returns:
      str: The cleaned text.
  """
  # Remove special characters, punctuations, numbers, and uppercase 'X'
  cleaned_text = re.sub(r'[^a-zA-Z\s]', '', text.replace('XXXX', '').replace('XX', ''))
  # Convert to lowercase
  cleaned_text = cleaned_text.lower()
  # Remove leading/trailing whitespaces
  return cleaned_text.strip()

# Apply cleaning to the 'Consumer complaint narrative' column
# Create a column named consumer_narrative for the cleaned narrative
df.loc[:, 'consumer_narrative'] = df['Consumer complaint narrative'].apply(clean_text)

# Print the cleaned DataFrame (optional)
print(df['consumer_narrative'])

0         my name is    this complaint is not made in er...
1         in accordance with the fair credit reporting a...
2         by the laws that uphold deemed as fcra  fair c...
3         i am inputting a new compliant in follow up fr...
4         mortgage concern  i received a mortgage in  of...
                                ...                        
542644    rent relief paid the outstanding balance of  o...
542645                         have the information removed
542646    upgrade continues to call me over and over mut...
542647    originally i tried to dispute with the credit ...
542648    on  i opened a smartly checking account online...
Name: consumer_narrative, Length: 542649, dtype: object


In [10]:
# Print unique values in the column
print(df['consumer_narrative'].unique())

['my name is    this complaint is not made in error neither is it being made by a third partyi declare under penalty of perjury i am alleging that a person or company without my authorization from unauthorized use of my social security number and card used my personal identifying information to apply for goods services or money  and was successful in creating some accounts i have no knowledge of this and all is being investigated by the ftc and under  us code b permissible purposes of consumer reports i never gave any written consent to report anything on my consumer reports no consent is fraud'
 'in accordance with the fair credit reporting act   account   has violated  usc  section  a states i have the right to privacy  usc  section  a section  it also states consumer reporting agency can not furnish a without my written instructions'
 'by the laws that uphold deemed as fcra  fair credit reporting act  usc the furnishers  credit reporting agencies of the following accounts inquirys  

### Tukenization


Tokenization is the process of breaking down a text document or a sequence of characters into smaller units called tokens, which could be words, phrases, symbols, or other meaningful elements. This facilitates tasks such as natural language processing, where understanding the individual components of a text is crucial for analysis, classification, or generation

In [11]:
# Apply word tokenization to the 'consumer_narrative' column
df['consumer_narrative'] = df['consumer_narrative'].apply(word_tokenize)

# Print the DataFrame with tokenized text (optional)
print(df['consumer_narrative'])

0         [my, name, is, this, complaint, is, not, made,...
1         [in, accordance, with, the, fair, credit, repo...
2         [by, the, laws, that, uphold, deemed, as, fcra...
3         [i, am, inputting, a, new, compliant, in, foll...
4         [mortgage, concern, i, received, a, mortgage, ...
                                ...                        
542644    [rent, relief, paid, the, outstanding, balance...
542645                    [have, the, information, removed]
542646    [upgrade, continues, to, call, me, over, and, ...
542647    [originally, i, tried, to, dispute, with, the,...
542648    [on, i, opened, a, smartly, checking, account,...
Name: consumer_narrative, Length: 542649, dtype: object


### Lemmatization

Lemmatization is the process of reducing words to their base or dictionary form, known as the lemma. Unlike stemming, which involves chopping off prefixes or suffixes to derive the root form of a word, lemmatization considers the morphological analysis of the word and maps it to its canonical form found in the dictionary. This ensures that different inflected forms of a word are treated as the same word, which aids in tasks like natural language understanding and text analysis.

In [12]:
# Create a WordNetLemmatizer object
lemmatizer = WordNetLemmatizer()

# Apply lemmatization to the 'consumer_narrative' column
df['consumer_narrative'] = df['consumer_narrative'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])

# Print the DataFrame with lemmatized text (optional)
print(df['consumer_narrative'])

0         [my, name, is, this, complaint, is, not, made,...
1         [in, accordance, with, the, fair, credit, repo...
2         [by, the, law, that, uphold, deemed, a, fcra, ...
3         [i, am, inputting, a, new, compliant, in, foll...
4         [mortgage, concern, i, received, a, mortgage, ...
                                ...                        
542644    [rent, relief, paid, the, outstanding, balance...
542645                    [have, the, information, removed]
542646    [upgrade, continues, to, call, me, over, and, ...
542647    [originally, i, tried, to, dispute, with, the,...
542648    [on, i, opened, a, smartly, checking, account,...
Name: consumer_narrative, Length: 542649, dtype: object


### Stopwords

Stopwords are common words in a language that are often removed from text data during preprocessing because they typically do not carry significant meaning or contribute to the understanding of the text. Examples of stopwords in English include "the," "is," "and," "in," "of," etc. Removing stopwords can help reduce noise and improve the performance of natural language processing tasks such as text classification, sentiment analysis, and topic modeling.

In [13]:
# Download stopwords dataset
nltk.download('stopwords')

# Define stop words
stop_words = set(stopwords.words('english'))

# Remove stop words from the 'Consumer complaint narrative' column
df['consumer_narrative'] = df['consumer_narrative'].apply(lambda tokens: [word for word in tokens if word.lower() not in stop_words])

# Display the updated DataFrame (optional)
print(df['consumer_narrative'])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\s4114805\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


0         [name, complaint, made, error, neither, made, ...
1         [accordance, fair, credit, reporting, act, acc...
2         [law, uphold, deemed, fcra, fair, credit, repo...
3         [inputting, new, compliant, follow, compliant,...
4         [mortgage, concern, received, mortgage, purcha...
                                ...                        
542644    [rent, relief, paid, outstanding, balance, emp...
542645                               [information, removed]
542646    [upgrade, continues, call, mutiple, time, one,...
542647    [originally, tried, dispute, credit, bureau, l...
542648    [opened, smartly, checking, account, online, u...
Name: consumer_narrative, Length: 542649, dtype: object


In [14]:
# Further Cleaning

def clean_text(text):
  """
  This function cleans text data by performing additional processing steps.

  Args:
      text (str or list): The text to be cleaned. Can be a string or a list of tokens.

  Returns:
      str: The cleaned text.
  """
  if isinstance(text, list):
    """
    - If the input is a list (likely from tokenization), join the elements
      into a single string for further cleaning.
    """
    text = ' '.join(text)  # Join list elements with spaces

  # Remove square brackets and commas
  cleaned_text = re.sub(r'[\[\],]', '', text)  # Regex to remove square brackets and commas

  # Remove extra spaces
  cleaned_text = ' '.join(cleaned_text.split())  # Split, remove extra spaces, rejoin with single space

  # Remove leading and trailing whitespaces
  return cleaned_text.strip()

# Apply the cleaning function to the entire column
df['consumer_narrative'] = df['consumer_narrative'].apply(clean_text)

# Print the cleaned data
print(df['consumer_narrative'])

0         name complaint made error neither made third p...
1         accordance fair credit reporting act account h...
2         law uphold deemed fcra fair credit reporting a...
3         inputting new compliant follow compliant citi ...
4         mortgage concern received mortgage purchase co...
                                ...                        
542644    rent relief paid outstanding balance employee ...
542645                                  information removed
542646    upgrade continues call mutiple time one day pa...
542647    originally tried dispute credit bureau late pa...
542648    opened smartly checking account online u bank ...
Name: consumer_narrative, Length: 542649, dtype: object


In [15]:
df

,Unnamed: 0,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID,consumer_narrative
0,0,04/25/23,"Credit reporting, credit repair services, or o...",Credit reporting,Improper use of your report,Reporting company used your report improperly,My name is XXXX XXXX XXXX this complaint is no...,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,MD,21206,None,Consent provided,Web,04/25/23,Closed with explanation,Yes,NaN,6880772,name complaint made error neither made third p...
1,1,07/10/23,"Credit reporting, credit repair services, or o...",Credit reporting,Improper use of your report,Reporting company used your report improperly,In accordance with the Fair Credit Reporting a...,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",OK,73115,None,Consent provided,Web,07/10/23,Closed with non-monetary relief,Yes,NaN,7229816,accordance fair credit reporting act account h...
2,2,01/10/24,Credit reporting or other personal consumer re...,Credit reporting,Incorrect information on your report,Account information incorrect,By the Laws that uphold deemed as FCRA ( Fair ...,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",CA,91606,None,Consent provided,Web,01/10/24,Closed with non-monetary relief,Yes,NaN,8139386,law uphold deemed fcra fair credit reporting a...
3,3,08/01/23,Checking or savings account,Checking account,Opening an account,Didn't receive terms that were advertised,I am inputting a new compliant in follow up fr...,Company has responded to the consumer and the ...,"CITIBANK, N.A.",TX,75126,None,Consent provided,Web,08/01/23,Closed with explanation,Yes,NaN,7334854,inputting new compliant follow compliant citi ...
4,4,02/27/23,Mortgage,Conventional home mortgage,Trouble during payment process,None,Mortgage Concern : I received a mortgage in XX...,None,SN SERVICING CORPORATION,DC,20003,None,Consent provided,Web,03/22/23,Closed with explanation,Yes,NaN,6625444,mortgage concern received mortgage purchase co...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
542644,542644,03/04/23,Debt collection,Mortgage debt,Attempts to collect debt not owed,Debt was paid,Rent relief paid the outstanding balance of XX...,None,"National Credit Systems,Inc.",TX,76040,None,Consent provided,Web,03/04/23,Closed with explanation,Yes,NaN,6646662,rent relief paid outstanding balance employee ...
542645,542645,04/06/23,Debt collection,Other debt,Attempts to collect debt not owed,Debt was already discharged in bankruptcy and ...,Have the information removed,None,CL Holdings LLC,SC,29360,None,Consent provided,Web,04/06/23,Closed with explanation,Yes,NaN,6803727,information removed
542646,542646,08/11/23,Debt collection,Credit card debt,Communication tactics,Frequent or repeated calls,Upgrade continues to call me over and over Mut...,None,"Upgrade, Inc.",FL,33065,None,Consent provided,Web,08/11/23,Closed with explanation,Yes,NaN,7389495,upgrade continues call mutiple time one day pa...
542647,542647,02/24/23,"Credit reporting, credit repair services, or o...",Credit reporting,Problem with a credit reporting company's inve...,Their investigation did not fix an error on yo...,Originally I tried to dispute with the credit ...,None,"EQUIFAX, INC.",KY,42141,None,Consent provided,Web,02/24/23,Closed with explanation,Yes,NaN,6606585,originally tried dispute credit bureau late pa...


In [17]:
df.drop('Consumer disputed?', axis=1, inplace=True)

In [18]:
# Count the number of missing values in each column of the DataFrame
df.isnull().sum()

Unnamed: 0                      0
Date received                   0
Product                         0
Sub-product                     0
Issue                           0
Sub-issue                       0
Consumer complaint narrative    0
Company public response         0
Company                         0
State                           0
ZIP code                        0
Tags                            0
Consumer consent provided?      0
Submitted via                   0
Date sent to company            0
Company response to consumer    0
Timely response?                0
Complaint ID                    0
consumer_narrative              0
dtype: int64

In [19]:
df.to_csv('my_data_Cleaned.csv') 